In [1]:
import numpy as np
import pandas as pd
import transformers
from transformers import pipeline
import seaborn as sns
import torch

In [2]:
df=pd.read_csv(r'C:\Users\ella.gorgin\Dropbox\Data_Analytics\Data_sets\crypto_tweets\crypto_10k_tweets_(2021_2022Nov).csv', lineterminator='\n')
df.head()

,Unnamed: 0,Date,Username,Content,URL,Hashtags
0,0,2022-11-30 11:53:21+00:00,0xEthereumYoda,#Ethereum price update: \n\n#ETH $1269.23 USD\...,https://twitter.com/0xEthereumYoda/status/1597...,"['Ethereum', 'ETH', 'Bitcoin', 'BTC', 'altcoin..."
1,1,2022-11-30 11:53:21+00:00,Lawrenc32984128,@mtmalinen @ecb Do well to understand that eve...,https://twitter.com/Lawrenc32984128/status/159...,['BTC']
2,2,2022-11-30 11:53:21+00:00,NITESHP55784410,Kayla #Ethereum Harold #世界杯 Egbert #百家乐 Lavern...,https://twitter.com/NITESHP55784410/status/159...,"['Ethereum', '世界杯', '百家乐', '世界杯买球']"
3,3,2022-11-30 11:53:20+00:00,817coin,#Bitcoin https://t.co/2koLlCvCri https://t.co/...,https://twitter.com/817coin/status/15979217174...,['Bitcoin']
4,4,2022-11-30 11:53:18+00:00,slamtoken,"Yesterday we made a 3,000 SLAM buyback and loc...",https://twitter.com/slamtoken/status/159792171...,"['slamtoken', 'bnb', 'bsc']"


In [3]:
df=df.drop('Unnamed: 0', axis=1)

In [4]:
df=df.head(10)

# pipeline

In [5]:
pipeline = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [6]:
strings = ["".join(i) for i in df['Content']]


In [8]:
from tqdm.notebook import tqdm
pip=[]
label=[]
for i, row in tqdm(df.iterrows(), total=len(df)):
        pipel = pipeline(row['Content'])
        pip.append(pipel[0]['score'])
        label.append(pipel[0]['label'])
    

  0%|          | 0/10 [00:00<?, ?it/s]

In [9]:
df['pip_label']=label
df['pip_score']=pip
df.head()

,Date,Username,Content,URL,Hashtags,pip_label,pip_score
0,2022-11-30 11:53:21+00:00,0xEthereumYoda,#Ethereum price update: \n\n#ETH $1269.23 USD\...,https://twitter.com/0xEthereumYoda/status/1597...,"['Ethereum', 'ETH', 'Bitcoin', 'BTC', 'altcoin...",NEGATIVE,0.990652
1,2022-11-30 11:53:21+00:00,Lawrenc32984128,@mtmalinen @ecb Do well to understand that eve...,https://twitter.com/Lawrenc32984128/status/159...,['BTC'],POSITIVE,0.999539
2,2022-11-30 11:53:21+00:00,NITESHP55784410,Kayla #Ethereum Harold #世界杯 Egbert #百家乐 Lavern...,https://twitter.com/NITESHP55784410/status/159...,"['Ethereum', '世界杯', '百家乐', '世界杯买球']",NEGATIVE,0.990453
3,2022-11-30 11:53:20+00:00,817coin,#Bitcoin https://t.co/2koLlCvCri https://t.co/...,https://twitter.com/817coin/status/15979217174...,['Bitcoin'],NEGATIVE,0.996596
4,2022-11-30 11:53:18+00:00,slamtoken,"Yesterday we made a 3,000 SLAM buyback and loc...",https://twitter.com/slamtoken/status/159792171...,"['slamtoken', 'bnb', 'bsc']",POSITIVE,0.993540


# Roberta

In [10]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
# model
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [11]:
from tqdm import tqdm 

tweet_words = []
neg=[]
neu=[]
pos=[]
for ind, tweet in tqdm(df.iterrows(), total=len(df)):    
    for word in tweet['Content'].split(' '):
        word = '@user' if word.startswith('@') and len(word) > 1 else word
        word = 'http' if word.startswith('http') else word
        word = 'hastag' if word.startswith('#') else word
        tweet_words.append(word)
    tweet_processed = " ".join(tweet_words)
    encoded_tweet = tokenizer(tweet_processed, return_tensors='pt')
    output = model(**encoded_tweet)
    scores = softmax(output[0][0].detach().numpy())  
    neg.append(scores[0])
    neu.append(scores[1])
    pos.append(scores[2])



100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.56it/s]


In [12]:
df['Roberta_Negative']=neg
df['Roberta_Neutral']=neu
df['Roberta_Positive']=pos
df.head()

,Date,Username,Content,URL,Hashtags,pip_label,pip_score,Roberta_Negative,Roberta_Neutral,Roberta_Positive
0,2022-11-30 11:53:21+00:00,0xEthereumYoda,#Ethereum price update: \n\n#ETH $1269.23 USD\...,https://twitter.com/0xEthereumYoda/status/1597...,"['Ethereum', 'ETH', 'Bitcoin', 'BTC', 'altcoin...",NEGATIVE,0.990652,0.042857,0.901389,0.055754
1,2022-11-30 11:53:21+00:00,Lawrenc32984128,@mtmalinen @ecb Do well to understand that eve...,https://twitter.com/Lawrenc32984128/status/159...,['BTC'],POSITIVE,0.999539,0.006791,0.193536,0.799673
2,2022-11-30 11:53:21+00:00,NITESHP55784410,Kayla #Ethereum Harold #世界杯 Egbert #百家乐 Lavern...,https://twitter.com/NITESHP55784410/status/159...,"['Ethereum', '世界杯', '百家乐', '世界杯买球']",NEGATIVE,0.990453,0.007306,0.224752,0.767941
3,2022-11-30 11:53:20+00:00,817coin,#Bitcoin https://t.co/2koLlCvCri https://t.co/...,https://twitter.com/817coin/status/15979217174...,['Bitcoin'],NEGATIVE,0.996596,0.007497,0.230922,0.761581
4,2022-11-30 11:53:18+00:00,slamtoken,"Yesterday we made a 3,000 SLAM buyback and loc...",https://twitter.com/slamtoken/status/159792171...,"['slamtoken', 'bnb', 'bsc']",POSITIVE,0.993540,0.006447,0.209190,0.784363
